In [4]:
%set_env CUDA_VISIBLE_DEVICES=0

import torch
import tqdm
import gpytorch
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood


env: CUDA_VISIBLE_DEVICES=0


In [5]:
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL


In [8]:
#we use Concrete dataset as an example
import urllib.request
import os
from scipy.io import loadmat
from math import floor

import numpy as np
import pandas as pd
np.random.seed(109)
torch.manual_seed(42)
import torch.nn as nn
data = pd.read_excel('Concrete_Data.xls')
data = torch.tensor(data.values).float()

X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]
y =(y - y.mean())/y.std()


shuffle_indices = np.random.permutation(len(X))
X_shuffled = X[shuffle_indices]
y_shuffled = y[shuffle_indices]


train_n = int(floor(0.9 * len(X)))
train_x = X_shuffled[:train_n, :].contiguous()
train_y = y_shuffled[:train_n].contiguous()

test_x = X_shuffled[train_n:, :].contiguous()
test_y = y_shuffled[train_n:].contiguous()


ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [ ]:
num_inducing=128
input_dims=train_x.shape[-1]

t1=0.01

class lsde(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        
        self.nn1= torch.nn.Sequential(
           nn.Linear(num_inducing, 128),
            nn.Tanh(),
           
            nn.Linear(128, num_inducing)
        ).to(device)
       
        
        
        self.s=1
  
        
        self.sigma=torch.nn.Parameter(torch.ones([1,num_inducing])).to(device)
        
        
        
        self.noise_type = "diagonal"
        self.sde_type = "ito"
    def lam(self,t):
        lamda=t1-(t+0.001)
        return lamda

    
    def f(self, t,x):
       
        lamda=self.lam(t)
        
        f= 2*self.sigma**2*self.nn1(x)+x
       
        return f*lamda
  
    def g(self,t,x):
        g=self.sigma
        
        return g
   


In [7]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

NameError: name 'train_x' is not defined

In [40]:

train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()


True


In [42]:
num_inducing=128
input_dims=train_x.shape[-1]



In [43]:

from gpytorch.distributions import MultivariateNormal
from gpytorch.lazy import CholLazyTensor, TriangularLazyTensor
from gpytorch.variational._variational_distribution import _VariationalDistribution

class CholeskyVariationalDistribution(_VariationalDistribution):


    def __init__(self,num_inducing_points, batch_shape=torch.Size([]), mean_init_std=1e-3, **kwargs):
        super().__init__(num_inducing_points=num_inducing_points, batch_shape=batch_shape, mean_init_std=mean_init_std)

        ##sde
        self.u0 =torch.nn.Parameter(torch.randn([1,num_inducing_points]).type(torch.float32)).to(device)
        
        
        self.usde = lsde().to(device)
        self.uT =self.u().to(device)


        self.mean_init =  self.uT 
       
        self.covar_init = torch.eye(num_inducing_points).to(device)  # prior: p(U)
        self.mean_init =  self.mean_init.repeat(*batch_shape, 1)  # [output_dim,num_inducing_points]
        self.covar_init =  self.covar_init.repeat(*batch_shape, 1, 1)  # [output_dim,num_inducing_points,num_inducing_points]
     

        # set mean and std of prior p(U) to trainable parameters
        self.register_parameter(name="variational_mean", parameter=torch.nn.Parameter(self.mean_init))
        self.register_parameter(name="chol_variational_covar", parameter=torch.nn.Parameter(self.covar_init))
       


    def forward(self):
       
        
        
        
      
        chol_variational_covar = self.chol_variational_covar
        dtype = chol_variational_covar.dtype
        device = chol_variational_covar.device

        # First make the cholesky factor is upper triangular
        lower_mask = torch.ones(self.chol_variational_covar.shape[-2:], dtype=dtype, device=device).tril(0)
        chol_variational_covar = TriangularLazyTensor(chol_variational_covar.mul(lower_mask))

        # Now construct the actual matrix
        variational_covar = CholLazyTensor(chol_variational_covar)
        return MultivariateNormal(self.variational_mean, variational_covar)

    def initialize_variational_distribution(self, prior_dist):
        self.variational_mean.data.copy_(prior_dist.mean)
        self.variational_mean.data.add_(torch.randn_like(prior_dist.mean), alpha=self.mean_init_std)  # add a noise
        self.chol_variational_covar.data.copy_(prior_dist.lazy_covariance_matrix.cholesky().evaluate())
    
    def u(self):
        t0=0.0
        
       
        ts=torch.linspace(t0, t1, steps=int(t1-t0)*1000+1)
       
        u_s= torchsde.sdeint(self.usde, y0=self.u0, ts=ts,names={'drift': 'f', 'diffusion': 'g'},method='euler',dt=0.001)
        u=u_s[-1,:,:]
       
        
        return u.squeeze()
    

In [44]:
import torch.nn as nn
import torchsde
device='cuda:0'

class ToyDeepGPHiddenLayer(DeepGPLayer):
    def __init__(self, input_dims, output_dims, num_inducing, mean_type='constant'):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims).to(device)
            batch_shape = torch.Size([])
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims).to(device)
            batch_shape = torch.Size([output_dims])

        variational_distribution = CholeskyVariationalDistribution(
            
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        ).to(device)

        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        ).to(device)

        super(ToyDeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)
        
        self.variational_distribution = CholeskyVariationalDistribution(
            
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        ).to(device)

        self.variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            self.variational_distribution,
            learn_inducing_locations=True
        ).to(device)


        if mean_type == 'constant':
            self.mean_module = ConstantMean(batch_shape=batch_shape).to(device)
        else:
            self.mean_module = LinearMean(input_dims).to(device)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape, ard_num_dims=None
        ).to(device)

    def forward(self, x):
        mean_x = self.mean_module(x).to(device)
        covar_x = self.covar_module(x).to(device)
        return MultivariateNormal(mean_x, covar_x)

    def __call__(self, x, *other_inputs, **kwargs):
        """
        Overriding __call__ isn't strictly necessary, but it lets us add concatenation based skip connections
        easily. For example, hidden_layer2(hidden_layer1_outputs, inputs) will pass the concatenation of the first
        hidden layer's outputs and the input data to hidden_layer2.
        """
        if len(other_inputs):
            if isinstance(x, gpytorch.distributions.MultitaskMultivariateNormal):
                x = x.rsample().to(device)

            processed_inputs = [
                inp.unsqueeze(0).expand(gpytorch.settings.num_likelihood_samples.value(), *inp.shape)
                for inp in other_inputs
            ]

            x = torch.cat([x] + processed_inputs, dim=-1).to(device)

        return super().__call__(x, are_samples=bool(len(other_inputs)))

In [45]:


num_output_dims =  input_dims


class DDeepGP(DeepGP):
    def __init__(self, train_x_shape=train_x.shape):
        hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_output_dims,
            num_inducing=num_inducing,
            mean_type='linear',
        ).to(device)
        hidden_layer1 = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_output_dims,
            num_inducing=num_inducing,
            mean_type='linear',
        ).to(device)
        
        last_layer = ToyDeepGPHiddenLayer(
            input_dims=hidden_layer1.output_dims,
            output_dims=None,
            num_inducing=num_inducing,
            mean_type='constant',
        ).to(device)
        
        super().__init__()
        
        self.hidden_layer = hidden_layer
        self.hidden_layer1 = hidden_layer1
        
        self.last_layer = last_layer
        self.likelihood = GaussianLikelihood()
    
    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        hidden_rep2 = self.hidden_layer1(hidden_rep1)
        output = self.last_layer(hidden_rep1)
        return output
    
    def predict(self, test_loader):
        with torch.no_grad():
            mus = []
            variances = []
            lls = []
            for x_batch, y_batch in test_loader:
                preds = self.likelihood(self(x_batch))
                mus.append(preds.mean)
                variances.append(preds.variance)
                lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))
        
        return torch.cat(mus, dim=-1), torch.cat(variances, dim=-1), torch.cat(lls, dim=-1)
       

In [13]:
num_output_dims =  input_dims
##dgp2

class DDeepGP(DeepGP):
    def __init__(self, train_x_shape=train_x.shape):
        hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_output_dims,
            num_inducing=num_inducing,
            mean_type='linear',
        ).to(device)
        
        last_layer = ToyDeepGPHiddenLayer(
            input_dims=hidden_layer.output_dims,
            output_dims=None,
            num_inducing=num_inducing,
            mean_type='constant',
        ).to(device)
        
        super().__init__()
        
        self.hidden_layer = hidden_layer
        self.last_layer = last_layer
        self.likelihood = GaussianLikelihood()
    
    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        output = self.last_layer(hidden_rep1)
        return output
    
    def predict(self, test_loader):
        with torch.no_grad():
            mus = []
            variances = []
            lls = []
            for x_batch, y_batch in test_loader:
                preds = self.likelihood(self(x_batch))
                mus.append(preds.mean)
                variances.append(preds.variance)
                lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))
        
        return torch.cat(mus, dim=-1), torch.cat(variances, dim=-1), torch.cat(lls, dim=-1)

In [46]:
model = DDeepGP()
if torch.cuda.is_available():
    model = model.cuda()

In [54]:
# this is for running the notebook in our testing framework

num_epochs = 500
num_samples = 1
model.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
], lr=0.01)
mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, train_x.shape[-2]))

epochs_iter = tqdm.tqdm(range(num_epochs), desc="Epoch")





for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = tqdm.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
       
        with gpytorch.settings.num_likelihood_samples(num_samples):
            optimizer.zero_grad()
            output = model(x_batch.to(device))
         
            loss1 = -mll(output, y_batch.to(device))
            ts=torch.linspace(0, t1, steps=int(t1-0)*1000+1).to(device)
            u0 =model.hidden_layer.variational_distribution.u0.to(device)
            
         
            g=model.hidden_layer.variational_distribution.usde.sigma.unsqueeze(0)**2
            u_s= torchsde.sdeint(model.hidden_layer.variational_distribution.usde.to(device), y0=u0, ts=ts,names={'drift': 'f', 'diffusion': 'g'},method='euler',dt=0.001).to(device)
            
            r=model.hidden_layer.variational_distribution.usde.nn1(u_s)
            loss2=0.5*torch.norm(g*(u_s+r))**2
            u1 =model.last_layer.variational_distribution.u0.to(device)
         
            g1=model.last_layer.variational_distribution.usde.sigma.unsqueeze(0)**2
            u_s1= torchsde.sdeint(model.last_layer.variational_distribution.usde.to(device), y0=u1, ts=ts,names={'drift': 'f', 'diffusion': 'g'},method='euler',dt=0.001).to(device)
            r1=model.last_layer.variational_distribution.usde.nn1(u_s+r)
            loss3=0.5*torch.norm(g1*(u_s1+r1))**2

            u2 =model.hidden_layer1.variational_distribution.u0.to(device)
         
            g2=model.hidden_layer1.variational_distribution.usde.sigma.unsqueeze(0)**2
            u_s2= torchsde.sdeint(model.hidden_layer1.variational_distribution.usde.to(device), y0=u2, ts=ts,names={'drift': 'f', 'diffusion': 'g'},method='euler',dt=0.001).to(device)
            
            r2=model.hidden_layer1.variational_distribution.usde.nn1(u_s2)
            loss4=0.5*torch.norm(g2*(u_s2+r2))**2

           
            
            
            
            
            
            
            loss=loss1+loss2+loss3+loss4
        
            loss.backward()
            optimizer.step()

            minibatch_iter.set_postfix(loss=loss1.item())
            optimizer.zero_grad()


Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.277]
                                                            
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.27]
                                                           
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.26]
                                                           
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.268]
                                                            
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.294]
                                                            
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.268]
                                                            
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.269]
                                                            
Minibatch:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.264]
                                                            
Minibatch:   0%|          | 

64.72488808631897


In [55]:
import math


test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024)

model.eval()

print(model.hidden_layer.variational_distribution)

CholeskyVariationalDistribution(
  (usde): lsde(
    (nn1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
  )
)


In [56]:
device='cuda:0'
rm=[]
for i in range(10):    
    pred=torch.tensor([]).to(device)
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch).mean
            pred=torch.cat((pred,preds),dim=0)
           
    rmse = torch.mean(torch.pow(pred - test_y, 2)).sqrt()
    rm.append(rmse)
print(f"RMSE: {rmse.mean().item()}")

RMSE: 0.2964940369129181
